### Libararies

In [1]:
import pandas as pd
import numpy as np 

import warnings
import os


from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")
os.chdir("C:\\Users\\joaos\\Documents\\GitHub\\pred_se_ml\data")

### Data Manipulation

In [4]:
# Importing the data
df = pd.read_csv("pred_se_ml.csv")

df.head()

,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,unemployment,tax_burden,diversity,quality,se_medina,democ_closed_anocracy,democ_democracy,democ_full_democracy,democ_interregnum,democ_interruption,democ_open_anocracy,democ_transition,lgdp_pc
0,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,NaN,61.6,6.002369,0.37,52.47,0,0,0,0,0,0,1,5.986754
1,2.762770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.531644,NaN,54.6,6.048760,0.40,46.30,0,0,0,0,0,0,1,6.118923
2,5.363360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.453022,NaN,52.6,5.991146,0.35,50.48,0,0,0,0,0,0,0,6.246355
3,17.121191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.359348,NaN,59.1,5.882152,0.34,49.21,0,0,0,0,0,0,0,6.048325
4,40.167251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557.501113,NaN,47.9,5.903543,0.34,48.64,0,0,0,0,0,0,0,5.960205


###

### Splitting in train and test datasets

Separating the final data set in train and test. The training data set will be used to train and validate the machine learning algorithms. 

In [6]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=0)